Inspired by [assistant-wind_farm.ipynb](https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/api-in-a-box/wind_farm/assistant-wind_farm.ipynb)

In [1]:
# Define the Assistant tools

tools_list = [
    {"type": "code_interpreter"},
]

In [2]:
# Libraries and Constants

import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../credentials_my.env")
MODEL = os.environ["GPT4-1106-128k"]

In [3]:
from openai import AzureOpenAI

In [4]:
# Create the client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview", # at least 2024-02-15-preview
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [5]:
# Upload the Assistant file(s)

from openai.types import FileObject
from pathlib import Path

DATA_FOLDER = "./data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    print(f"Uploading {filePath}...")
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]
assistant_files

Uploading ./data/turbines.csv...


[FileObject(id='assistant-eUc74kzlFd8ZzWAPM9uIXm5g', bytes=576, created_at=1710721659, filename='turbines.csv', object='file', purpose='assistants', status='processed', status_details=None)]

In [6]:
# Create an Assistant and a Thread
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are an assistant that can help manage wind turbine farm. "
    + "The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour."
    + "Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
    tools=tools_list,
    model=MODEL,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

print(assistant.model_dump_json(indent=2))

{
  "id": "asst_VInZztTArxbm4vQaeRIMzAoi",
  "created_at": 1710721660,
  "description": null,
  "file_ids": [
    "assistant-eUc74kzlFd8ZzWAPM9uIXm5g"
  ],
  "instructions": "You are an assistant that can help manage wind turbine farm. The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
  "metadata": {},
  "model": "gpt4-1106-128k",
  "name": "Portfolio Management Assistant",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


In [7]:
def process_message(content: str) -> None:
    from datetime import datetime
    import time, json
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ". ",
    )

    print("Processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed" or run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print(json.loads(messages.json())["data"][0]["content"][0]["text"]["value"])            
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            pass
        else:
            print(f"Status: {run.status}")
            time.sleep(5)
            
    return messages

In [8]:
r1 = process_message("What is the status of turbine 1001 and 1003?")

Processing...
Status: queued
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
The status for turbines 1001 and 1003 is as follows:

- Turbine 1001: Wind speed of 32, RPM of 18, voltage of 33, and a maintenance date scheduled for 2023-10-12.
- Turbine 1003: Wind speed of 30, RPM of 16, voltage of 32, and a maintenance date scheduled for 2023-11-30.

Keep in mind that this information might represent the latest recorded data or the scheduled maintenance and may not reflect real-time status, depending on the nature and timing of the data collection in the file. If you need further analysis or information, please let me know.


In [9]:
r2 = process_message("Show me the code you used to provide the previous answer")

Processing...
Status: queued
Status: in_progress
Status: in_progress
Status: in_progress
Certainly! Here is the Python code I used to extract the status of turbines 1001 and 1003 from the CSV file you provided:

```python
import pandas as pd

# Read the CSV file into a pandas DataFrame
file_path = '/mnt/data/assistant-eUc74kzlFd8ZzWAPM9uIXm5g'
df = pd.read_csv(file_path)

# Filter the DataFrame for the specific Turbine_IDs (1001 and 1003)
status_1001_1003 = df[df['Turbine_ID'].isin([1001, 1003])]

# Display the filtered data
status_1001_1003
```

Executing this code snippet in a Python environment would output the following DataFrame, showing the status for turbines 1001 and 1003:

```
   Turbine_ID  Wind_Speed  RPM  Voltage Maintenance_Date
1        1001          32   18       33       2023-10-12
3        1003          30   16       32       2023-11-30
```

This code assumes that the CSV file content is properly formatted and that the columns are named as expected. The `isin` method i

In [10]:
# Cleaning up

if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in client.files.list():
        print(f"Deleting the file {file.filename} (id={file.id})")
        client.files.delete(file.id)

Deleting the file turbines.csv (id=assistant-eUc74kzlFd8ZzWAPM9uIXm5g)
Deleting the file turbines.csv (id=assistant-OuIzH7uBajwlg1h4zHoLDa9G)


Bad pipe message: %s [b'T\x18O\xe4\x10\x9c\xaas\x0b\x8c^v\xf7*@,\xd7']
Bad pipe message: %s [b"\x8f\n\xd8\xf5\xb6&R\xb1d\x97\x00\xf6!n\xff\xbb\x91\xe0\\O['#\x05\xb5E\xef\xe8\xdaLf\xdd\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x9dn\x89\xeb\\0\xcd\xa6\xe8\xae\xfc\x15\xe0\xbaY\xa9\xce\xa8"]
Bad pipe message: %s [b"-\x9f\x19/G\xbf\xe8\x8c\xcf$\xc7%,\x1fG\x9eP+\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14